# The Boston Housing Dataset
## Modelos de regresión

## 1. Preámbulo/encabezado

In [ ]:
# vector, numeric, and visualization
import numpy as np
import matplotlib.pyplot as plt

# ml
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## 2. Base de datos

Ruta a la base de datos o repositorio, por facillidad se asume en el mismo directorio de este notebook.

In [ ]:
dataset_name = './boston.csv'

## 3. Carga de datos

Este comando carga en memoria una tabla de datos numéricos desde un archivo, asumiendo que los valores están separados por comas. Además, ignora la primera línea del archivo, que normalmente contiene los nombres de las columnas.

In [ ]:
datos = np.loadtxt(dataset_name, delimiter=',', skiprows=1)

El siguiente fragmento abre el archivo en modo lectura, toma únicamente la primera línea (que normalmente contiene los nombres de las columnas), elimina espacios o saltos de línea al inicio y al final, y luego la muestra en pantalla.

In [ ]:
with open(dataset_name, "r", encoding="utf-8") as f:
    primera_linea = f.readline().strip()

nombres_columnas = primera_linea.split(",")  # vector/lista con los nombres
print("Nombres de columnas:", nombres_columnas)

**Recuerde verificar la integridad de los datos**. Esto lo puede realizar simplemente imprimiendo algunos vaolores y comparandolos con los almacenados en el archivo .csv

In [ ]:
print("Shape:", datos.shape)       # tamaño por dimensión (filas, columnas)

## 4. Modelo de regresión

### 4.1 División de los datos (entrenamiento/validación)

In [ ]:
p_train = 0.7

n = datos.shape[0]              # número de muestras
k = int(round(n * p_train))     # número exacto de muestras a usar

idx = np.random.choice(n, size=k, replace=False)  # siempre distintos (en cada ejecución)
is_train = np.zeros(n, dtype=bool)

is_train[idx] = True            # datos para entrenamiento
is_validation = ~is_train       # datos para validación

### 4.2 Selección de la característica(s) a modelar

In [ ]:
input_features = 0  # <-- Modifique
output_feature = 13 # <-- no modificar

# resultados
print("Característica(s) de entrada:", nombres_columnas[input_features])
print("Características de salida:", nombres_columnas[output_feature])

### 4.3 Modelo de regresión

In [ ]:
# Selección del orden del polinomio
orden = 1 # <-- Modifique

# scikit-learn requiere X en formato 2D: (n_muestras, n_características)
X_train = datos[is_train,input_features].reshape(-1, 1)
y_train =  datos[is_train,output_feature]

# Pipeline: características polinómicas + regresión lineal
modelo = make_pipeline(
    PolynomialFeatures(degree=orden, include_bias=False),
    LinearRegression()
)

# Entrenamiento
modelo.fit(X_train, y_train)

### 4.4 Parámetros del modelo

In [ ]:
# Parámetros del modelo: v(m) = b0 + b1*m + b2*m^2 + ... + b_orden*m^orden
lin = modelo.named_steps["linearregression"]

b0 = lin.intercept_
b = lin.coef_  # [b1, b2, ..., b_orden] porque include_bias=False

# imprime coeficientes
print(f"Orden: {orden}")
print(f"b0: {b0:.6f}")
for i, bi in enumerate(b, start=1):
    print(f"b{i}: {bi:.6e}")

### 4.5 Evaluación/validación del modelo

**Error en el conjunto de entrenamiento**

In [ ]:
y_hat_train = modelo.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_hat_train))
r2_train = r2_score(y_train, y_hat_train)

# RMSE: error cuadrático medio (en voltios).
# R^2: coeficiente de determinación (0 a 1).
print(f"RMSE (entrenamiento): {rmse_train:.6f}")
print(f"R^2  (entrenamiento): {r2_train:.6f}")

**Error en el conjunto de validación**

In [ ]:
X_validation = datos[is_validation, input_features].reshape(-1, 1)
y_validation =  datos[is_validation, output_feature]

y_hat_validation = modelo.predict(X_validation)
rmse_validation = np.sqrt(mean_squared_error(y_validation, y_hat_validation))
r2_validation = r2_score(y_validation, y_hat_validation)

# RMSE: error cuadrático medio (en voltios).
# R^2: coeficiente de determinación (0 a 1).
print(f"RMSE (validación): {rmse_validation:.6f}")
print(f"R^2  (validación): {r2_validation:.6f}")

## 5. Visualización de resultados

In [ ]:
# Respuesta del modelo en el dominio de los datos
x = np.linspace(datos[:,input_features].min(), datos[:,input_features].max(), 10).reshape(-1, 1)
y_regresion = modelo.predict(x)

In [ ]:
plt.figure(figsize=(10,5))
# datos de entrenamiento
plt.scatter(datos[is_train,input_features], datos[is_train,output_feature], c='blue', label='Entrenamiento')
# datos de validación
plt.scatter(datos[is_validation,input_features], datos[is_validation,output_feature], c='red', label='Validación')
# Curva de regresión
plt.plot(x.ravel(), y_regresion, '-', label='Regresión')
plt.grid(True)

plt.title('Datos y curva de regresión')
plt.xlabel(nombres_columnas[input_features])
plt.ylabel(nombres_columnas[output_feature])
plt.legend()
plt.show()

## 6. Conclusiones

A partir de la gráfica de CRIM vs MEDV y la recta de regresión, se pueden concluir que:

- Relación inversa: a mayor CRIM (tasa de crimen) baja MEDV (valor medio de la vivienda) (pendiente negativa).

- El modelo lineal “se estira” por unos pocos puntos extremos: existen valores muy altos de CRIM (≈40–90) con pocos ejemplos (outliers), los cuales tiran la recta hacia abajo y hacen que la predicción sea poco realista en ese rango (incluso llega a valores negativos de MEDV en la recta, que no tienen sentido).

- Alta dispersión para CRIM bajos: cuando CRIM está cerca de 0–5, MEDV toma muchos valores distintos (desde bajos hasta 50). Eso sugiere que CRIM por sí sola explica poco del precio, faltan variables importantes.

- Heterocedasticidad (varianza no constante): la nube no tiene “grosor” uniforme; el error cambia según CRIM. Esto viola supuestos típicos de regresión lineal simple y afecta la calidad del ajuste.

- Sesgo en MEDV: se puede observar una acumulación en 50 (clásico del Boston Housing). Eso introduce censura/saturación y puede sesgar el ajuste.

- Train y validación se comportan parecido: visualmente, los puntos rojos y azules siguen un patrón similar; no se ve un “desfase” enorme entre ambos conjuntos (buena señal de consistencia), aunque el ajuste sigue siendo pobre.

Mejoras recomendadas (si el objetivo es predecir mejor):

- Probar transformación: log(CRIM + 1) (reduce el efecto de outliers).

- Usar un modelo robusto (Huber/RANSAC) o recortar/winsorizar extremos.

- Pasar a regresión múltiple (agregar variables como RM, NOX, LSTAT, etc.).

- Probar otros modelos.